In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

In [2]:
cid ='d888370f9c2a4514aade84a1b6ae976e'
secret ='01774fdc665e4dcc8dc8c48667e46bf1'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
from requests import exceptions
import lyricsgenius as lg
import re
import requests

token = '0GGXQLMawu2kxxYNqTGFT79I0wq8McYb8jbOz0HGOQc-sCAmu0HuYKo_C71dsRg-'
genius = lg.Genius(token, retries=3)
genius.verbose = True
# Remove section headers (e.g. [Chorus]) from lyrics when searching
genius.remove_section_headers = True

# Include hits thought to be non-songs (e.g. track lists)
genius.skip_non_songs = True

# Exclude songs with these words in their title
genius.excluded_terms = ["(Remix)", "(Live)"]

In [4]:
l = ['7eX5SypK35V8Y9d9pS6rWy',
        '4fPBB44eDH71YohayI4eKV',
        #'2M7UdnD0fEaryh8TnCvqFX',
        #'6LqZFfv4fUP7va14Y6VW9a',
]

print(sp.audio_features(tracks=l))

np.random.seed(3407)
R = np.random.normal(0, 1, (25, 384))

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('msmarco-MiniLM-L-6-v3')

[{'danceability': 0.472, 'energy': 0.746, 'key': 11, 'loudness': -4.816, 'mode': 1, 'speechiness': 0.0321, 'acousticness': 0.00581, 'instrumentalness': 0.663, 'liveness': 0.0999, 'valence': 0.403, 'tempo': 97.44, 'type': 'audio_features', 'id': '7eX5SypK35V8Y9d9pS6rWy', 'uri': 'spotify:track:7eX5SypK35V8Y9d9pS6rWy', 'track_href': 'https://api.spotify.com/v1/tracks/7eX5SypK35V8Y9d9pS6rWy', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7eX5SypK35V8Y9d9pS6rWy', 'duration_ms': 162040, 'time_signature': 4}, {'danceability': 0.63, 'energy': 0.908, 'key': 11, 'loudness': -2.42, 'mode': 1, 'speechiness': 0.0326, 'acousticness': 0.0238, 'instrumentalness': 0.592, 'liveness': 0.116, 'valence': 0.968, 'tempo': 120.522, 'type': 'audio_features', 'id': '4fPBB44eDH71YohayI4eKV', 'uri': 'spotify:track:4fPBB44eDH71YohayI4eKV', 'track_href': 'https://api.spotify.com/v1/tracks/4fPBB44eDH71YohayI4eKV', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4fPBB44eDH71YohayI4eKV', 'durat

KeyboardInterrupt: 

In [5]:
embed_remover = re.compile(r'\D*')

def remove_song_anatomy_tags(l):
    l = l.split('\n')
    l = l[1::]
    edited = []

    last_line = l[-1]
    last_line = last_line.split(' ')
    last_line[-1] = re.findall(embed_remover, last_line[-1])[0]
    l[-1] = ' '.join(last_line)

    for i in l:
        if i == '':
            continue

        if i == ' ':
            continue

        edited.append(i)

    return ' '.join(edited)


def get_song_lyrics_and_features(track_id, track_name, artist_name, popularity=50, explicit=0):
    feature_list = []
    track_features = sp.audio_features(tracks = [track_id])[0]

    feature_names = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'duration_ms']

    for feature in feature_names:
        feature_list.append(track_features[feature])
        
    feature_list.append(popularity)
    feature_list.append(explicit)
    print(feature_list)

    
    max_tries = 4

    for i in range(max_tries):

        try:
            song = genius.search_song(track_name, artist_name)
            break
        except requests.exceptions.Timeout:
            print(f'{track_name} pooped itself: {i} tries')
            if i == max_tries - 1:
                song = 'blank'
                raise

    print(song.lyrics)
    embeddings = embedder.encode(remove_song_anatomy_tags(song.lyrics), show_progress_bar=True)
    embeddings_proj = np.dot(R, embeddings)

    feature_list.extend(embeddings_proj)
    print(feature_list)
    return feature_list


# objs = list(bucket.objects.filter(Prefix=track_id))
#     if len(objs) > 0 and objs[0].key == track_id:
#         print(f"{track_name} is already in the S3 bucket.")
#         continue

In [6]:
playlist_id = 'https://open.spotify.com/playlist/2AXUGe93UP8oVj3F0a1Hoo?si=836ee38e473943a6'
results = sp.playlist_tracks(playlist_id)

# print(results)
for item in results['items'][:3]:
    track = item['track']
    track_id = track['id']
    track_name = track['name']
    artist_name = track['artists'][0]['name']
    popularity = track['popularity']
    explicit = int(track['explicit'])
    print(track_name, artist_name)
    
    # get_song_lyrics_and_features(track_id, track_name, artist_name, popularity, explicit)
    print(track_id)



Don't Let the Green Grass Fool You Wilson Pickett
[0.573, 0.707, 6, -8.29, 0, 0.0632, 0.442, 0.000241, 0.123, 0.961, 152.467, 4, 169707, 51, 0]
Searching for "Don't Let the Green Grass Fool You" by Wilson Pickett...
Done.
5 ContributorsDon’t Let the Green Grass Fool You Lyrics
Girl, try to remember
When we didn't have no shoes
We stuck together, just me and you
It took a long time to get what we got today
Now you wanna give it all up for another guy

Baby, I'm tellin' you
Don't let the green grass fool ya
Don't let it change your mind

It's gonna be hard for me, baby
To build my life all over
Because baby, you got the love
That I really need, really
'Cause if you leave
Leave me this way, baby
You know what?
I may not live to see a brand new day

Oh baby, don't let the green grass fool ya
Don't let it change your mind
You know why?
It may be greener on the other side, baby
But right in my arms is where you belong
You might also like
Mmm, ooh baby, oh, oh, ah

Don't let the green grass f

NameError: name 'embedder' is not defined

In [4]:
np.random.seed(3407)
R = np.random.normal(0, 1, (25, 384))

print(R)

[[ 1.34150514 -0.83548739  0.96262807 ...  1.51351945  1.48396066
   1.7828992 ]
 [ 0.87754169 -0.04139728  0.4215278  ... -0.12759557 -0.32730142
   0.63505085]
 [ 0.98795262 -0.25224437  0.86820121 ... -1.44767935  2.30604239
  -0.3456529 ]
 ...
 [-1.22244576 -0.45272609 -1.74057708 ... -0.25257253  1.04105811
   0.5675044 ]
 [-2.47316942  1.99770204  0.17841156 ...  1.40077656 -0.45673777
   1.40450841]
 [ 0.04512419 -1.10929368  0.89201495 ... -0.13570963 -0.19962704
  -1.91763652]]


In [7]:
' '.join(['fire', 'lol'])

'fire lol'

In [4]:
playlist_url = 'https://open.spotify.com/playlist/2AXUGe93UP8oVj3F0a1Hoo?si=836ee38e473943a6'
playlist = sp.playlist(playlist_url)
playlist_image_url = playlist['images'][0]['url']
print(playlist_image_url)

https://mosaic.scdn.co/640/ab67616d0000b2731bff3b5284c581b83e918d19ab67616d0000b273369abd4e740707a20132405eab67616d0000b27354dbcad8ae25bba3e552c537ab67616d0000b27391e6b5d4c6ffec6a50adda3f
